In [1]:
import pandas as pd
import numpy as np
import gc
import timeit
from numpy.linalg import inv

In [2]:
start_time = timeit.default_timer()
# df = pd.read_csv("/datasets/iowaliquor.csv")
df = pd.read_csv("../data_proj1/iowaliquor.csv")
df = df.iloc[::10, :]

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60)

df.head()

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Time (minutes) elapsed for this cell: 15.7295897877


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77.0,Polk,...,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,11.50,17.25,1,17.25,0.5,0.13
10,S05372600066,05/03/2012,2527,Hy-Vee Food Store #5 / Des Moines,1107 SE ARMY POST RD,DES MOINES,50315,POINT (-93.602372 41.526289),77.0,Polk,...,43026,Admiral Nelson Spiced Rum,12,750,5.77,8.66,24,207.84,18.0,4.76
20,S07363200013,08/28/2012,2662,Hy-Vee Wine & Spirits / Muscatine,"522 MULBERRY, SUITE A",MUSCATINE,52761,POINT (-91.04437 41.426615),70.0,Muscatine,...,37418,Popov Vodka 80 Prf,6,1750,8.49,12.74,12,152.88,21.0,5.55
30,S18612600022,04/24/2014,3437,Scott's Foods,1120 SUNSET DRIVE,NORWALK,50211,POINT (-93.67803700000002 41.479944),91.0,Warren,...,54646,Arrow Blackberry Flav Brandy,12,750,5.38,8.06,12,96.72,9.0,2.38
40,S21013800004,09/03/2014,2596,Hy-Vee / Regal Liquors and Video,2453 NORTH COURT,OTTUMWA,52501,POINT (-92.408085 41.049854),90.0,Wapello,...,42141,UV Sugar Crush Mini,1,3000,29.30,43.95,1,43.95,3.0,0.79


In [5]:
# DATA CLEANING/ORGANIZATION
start_time = timeit.default_timer()

# only include the features we need
df = df[["Date", "County", "Pack", "State Bottle Retail", 
         "Bottles Sold", "Sale (Dollars)", "Volume Sold (Gallons)"]]

df.dropna(inplace=True)

# condensing number of alcohol categories
df['Is_WHISKEY'] = df['Category Name'].str.contains('WHISK')
df['Is_VODKA'] = df['Category Name'].str.contains('VODKA')
df['Is_RUM'] = df['Category Name'].str.contains('RUM')
df['Is_SCHNAPPS'] = df['Category Name'].str.contains('SCHNAPPS')
df['Is_BRANDY'] = df['Category Name'].str.contains('BRAND')
df['Is_GIN'] = df['Category Name'].str.contains('GIN')
df["Is_TEQUILA"] = df['Category Name'].str.contains('TEQUILA')
df['Consolidated'] = pd.Series()
#df["Consolidated"] = ""

df.loc[df['Is_WHISKEY'] == True, 'Consolidated'] = 'Whiskey'
df.loc[df['Is_VODKA'] == True, 'Consolidated'] = 'Vodka'
df.loc[df['Is_RUM'] == True, 'Consolidated'] = 'Rum'
df.loc[df['Is_SCHNAPPS'] == True, 'Consolidated'] = 'Schnapps'
df.loc[df['Is_BRANDY'] == True, 'Consolidated'] = 'Brandy'
df.loc[df['Is_GIN'] == True, 'Consolidated'] = 'Gin'
df.loc[df['Is_TEQUILA'] == True, 'Consolidated'] = 'Tequila'
df.loc[df['Consolidated'] == "",'Consolidated'] = 'Other'

df['County'] = df['County'].str.lower()
df.loc[df["County"]=="buena vist","County"] = "buena vista"
df.loc[df["County"]=="cerro gord","County"] = "cerro gordo"
df.loc[df["County"]=="obrien","County"] = "o'brien"
df.loc[df["County"]=="pottawatta","County"] = "pottawattamie"
df["County"] = df["County"].str.capitalize()

# type casting
categorical_cols = ["Consolidated", "Vendor Name"]
for col in categorical_cols:
    df[col] = df[col].astype('category')
    
# get year from date
df['Date'] = pd.to_datetime(df['Date'])
df.insert(1, 'Year', df['Date'].dt.year)

# get dummies for categorical variables
county_var = df["County"]
#df = pd.get_dummies(df.drop(["County"], axis=1))
df["County"] = county_var

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60) 

<ipython-input-5-814511bca550>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


KeyError: 'Category Name'

In [ ]:
# AGGREGATION BY COUNTY AND YEAR
start_time = timeit.default_timer()

df.insert(1, 'Month-Year', df['Date'].dt.strftime('%m-%Y'))
df.drop(["Date", "Year"], axis=1, inplace=True)
county_var = df["County"]
month_year = df["Month-Year"]
df = pd.get_dummies(df.drop(["County", "Month-Year"], axis=1))
df["County"] = county_var
df["Month-Year"] = month_year

df.drop(["Consolidated_<class 'pandas.core.series.Series'>"], axis=1, inplace=True)

In [ ]:
df = df.groupby(['County', 'Month-Year']).agg('mean')

# add the year column back to the data frame
df.reset_index(level=["County", "Month-Year"], inplace=True)
df.insert(1, "Year", df["Month-Year"].str.replace("..-", ""))
df["Year"] = df["Year"].astype(int)

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60) 

df.head()

In [ ]:
# EXTERNAL DATASET, IOWA POPULATION FOR (county, year) PAIRS
df2 = pd.read_excel("iowa_county.xlsx")

# data cleaning/organization
df2 = df2.iloc[4:103]
df2.drop(["Unnamed: 1", "Unnamed: 2"], axis=1, inplace=True)
df2.rename(columns=
{"table with row headers in column A " +
 "and column headers in rows 3 through " +
 "4 (leading dots indicate sub-parts)": "County",
                    "Unnamed: 3": "2010", 
                    "Unnamed: 4": "2011", 
                    "Unnamed: 5": "2012", 
                    "Unnamed: 6": "2013", 
                    "Unnamed: 7": "2014",
                    "Unnamed: 8": "2015",
                    "Unnamed: 9": "2016",
                    "Unnamed: 10": "2017",
                    "Unnamed: 11": "2018",
                    "Unnamed: 12": "2019"}, inplace=True)
df2.reset_index(drop=True, inplace=True)
     
df2["County"] = df2["County"].str.replace(".", "")
df2["County"] = df2["County"].str.replace(" County, Iowa", "")

# convert df to standard form
df2 = df2.melt(var_name="Year", value_name="Population", id_vars=['County'])

# type casting
df2["County"] = df2["County"].astype('category')
df2["Year"] = df2["Year"].astype('int')
df2["Population"] = df2["Population"].astype('int')

df2.head()

In [ ]:
# EXTERNAL DATASET, IOWA INCOME PER CAPITA FOR (county, year) PAIRS
df_income = pd.read_csv("income.csv")

df_income = df_income[["Name", "Variable", "Value", "Date"]]
df_income = df_income[df_income["Variable"] == "Per capita personal income"]


df_income.rename(columns={"Name": "County", "Value": "Income Per Capita"}, inplace=True)

df_income["County"] = df_income["County"].str.replace(", IA", "")

df_income['Date'] = pd.to_datetime(df_income['Date'])
df_income["Year"] = df_income["Date"].dt.year

df_income.drop(["Variable", "Date"], axis=1, inplace=True)
df_income.head()

In [ ]:
# MERGE BOTH EXTERAL DATASETS
start_time = timeit.default_timer()

# merge with population values for each (county, year) pair
df = pd.merge(df, df2, on=["County", "Year"])

# per capita normalization
df.insert(6, "Bottles Sold Per Capita", df["Bottles Sold"]/df["Population"])
df.insert(7, "Alcohol Expense Per Capita", df["Sale (Dollars)"]/df["Population"])
df.insert(8, "Volume Sold (Gallons) Per Capita", df["Volume Sold (Gallons)"]/df["Population"])

cols_to_drop = ["Bottles Sold", "Sale (Dollars)", "Volume Sold (Gallons)"]
df.drop(cols_to_drop, axis=1, inplace=True)

# merge with income values for each (county, year) pair
df = pd.merge(df, df_income, on=["County", "Year"])
df.drop(["Year"], axis=1, inplace=True)

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60) 
df.head()

In [ ]:
iowa_month_county = df
iowa_month_county.to_csv("iowa_month_county.csv", index=False)

In [ ]:
iowa_month_county.head()